In [2]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Lambda, Input, AveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
#from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
import tensorflow as tf 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
import random
#import cv2
import os

In [4]:
image_names=list(os.listdir("Database"))
image_names.sort()
print(image_names)

['.DS_Store', 'COVID19', 'NORMAL', 'Viral_Pneumonia']


In [5]:
covid_dir=os.path.join("Database/COVID19")
norm_dir=os.path.join("Database/NORMAL")
pneu_dir=os.path.join("Database/Viral_Pneumonia")


print('total Covid images:', len(os.listdir(covid_dir)))
print('total Normal images:', len(os.listdir(norm_dir)))
print('total Viral_Pneumonia:', len(os.listdir(pneu_dir)))


total Covid images: 1327
total Normal images: 1341
total Viral_Pneumonia: 1463


In [6]:
df =pd.DataFrame(os.listdir(covid_dir))
df["condition"]= "Covid19"
df.head()

,0,condition
0,COVID-19 (979).png,Covid19
1,COVID-19 (580).png,Covid19
2,COVID-19 (996).png,Covid19
3,COVID-19 (646).png,Covid19
4,COVID-19 (216).png,Covid19


In [7]:
df =pd.DataFrame.from_records({"file_name":os.listdir(covid_dir),"condition": "Covid19"})

for f in image_names[1:]:
    folder_path="Database/" + f
    temp_df= pd.DataFrame.from_records({"file_name":os.listdir(folder_path),"condition": f })
    df=df.append(temp_df)
    
df["condition"].value_counts()

Viral_Pneumonia    1463
NORMAL             1341
Covid19            1327
COVID19            1327
Name: condition, dtype: int64

In [8]:
img_width, img_height=200,200
batch_size=128

data_dir = "Database/"


datagen= ImageDataGenerator(rescale=1/255,validation_split=.3,rotation_range=20,
                           shear_range=.2,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2)


train_generator= datagen.flow_from_directory(
                data_dir,
                target_size=(img_width,img_height),
                batch_size=batch_size,
                subset="training",
                class_mode="categorical", 
                classes= ["COVID19", "NORMAL","Viral_Pneumonia"],
                shuffle=True, seed=30)


datagen2=ImageDataGenerator(rescale=1/255,validation_split=.3)

test_generator=datagen2.flow_from_directory(
                data_dir,
                target_size=(img_width,img_height),
                batch_size=batch_size,
                classes= ["COVID19", "NORMAL","Viral_Pneumonia"],
                class_mode="categorical", subset="validation", shuffle=True, seed=30)



Found 2893 images belonging to 3 classes.
Found 1238 images belonging to 3 classes.


## Model 1
### Using new dataset and with 3 categories

In [9]:


classifier=Sequential()
classifier.add(Conv2D(32, kernel_size=3, activation="relu", input_shape=(200,200,3)))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(strides=(2,2)))
classifier.add(Dropout(.3))

classifier.add(Conv2D(32, kernel_size=3, activation="relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(strides=(2,2)))
classifier.add(Dropout(.5))

classifier.add(Conv2D(64, kernel_size=3, activation="relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(strides=(2,2)))
classifier.add(Dropout(.4))

classifier.add(Conv2D(64, kernel_size=3, activation="relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(strides=(2,2)))
classifier.add(Dropout(.3))

classifier.add(Flatten())
classifier.add(Dense(512,activation="relu"))
classifier.add(Dense(128,activation="relu"))
classifier.add(Dropout(.4))

classifier.add(Dense(3, activation="softmax"))

2022-01-19 11:46:12.955337: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 198, 198, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 99, 99, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 97, 97, 32)       1

In [ ]:
classifier.compile(loss="categorical_crossentropy", 
              optimizer="adam", 
              metrics=["categorical_accuracy"])

total_sample=train_generator.n
total_test=test_generator.n
batch_size=128

history= classifier.fit_generator(
            train_generator, steps_per_epoch=int(total_sample/batch_size),
            epochs=30,
            validation_data=test_generator, 
            validation_steps=int(total_test/batch_size))

In [ ]:
classifier.save("model_1.h5")

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('model_1.h5')

train_loss, train_acc = classifier.evaluate(train_generator)
print("\n Train Accuracy:", train_acc)
print("\n Train Loss:", train_loss)

In [ ]:
loaded_model = pickle.load(open(model_1.h5, 'rb'))
rtrain_loss, train_acc = loadedmodel.evaluate(train_generator)
print("\n Train Accuracy:", train_acc)
print("\n Train Loss:", train_loss)

In [ ]:
test_loss, test_acc= classifier.evaluate(test_generator)
print("\n Test Accuracy:", test_acc)
print("\n Test Loss:", test_loss)

In [ ]:
classifier.metrics_names

In [ ]:
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('Model Accuracy per Epoch')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower center')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Accuracy per Epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower center')
plt.show()

In [18]:
# Save history of model into csv
hist_df1 = pd.DataFrame(history.history) 

# or save to csv: 
hist_csv_file = 'history1.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df1.to_csv(f)

NameError: name 'history' is not defined

## Model 2

In [19]:
classifier= Sequential()

classifier.add(Conv2D(32, (3,3), input_shape=(200,200,3),activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(64,(3,3),activation="relu"))
classifier.add(Dropout(0.1))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(64, (3,3), activation="relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(128, (3,3), activation="relu"))
classifier.add(Dropout(0.2))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(128, (3,3), activation="relu"))
classifier.add(Dropout(0.2))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Flatten())


classifier.add(Dense(units = 128 , activation = 'relu'))
classifier.add(Dropout(0.2))

classifier.add(Dense(units = 3 , activation = 'softmax'))

In [22]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 198, 198, 32)      896       
                                                                 
 batch_normalization_4 (Batc  (None, 198, 198, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 99, 99, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 dropout_5 (Dropout)         (None, 97, 97, 64)        0         
                                                                 
 batch_normalization_5 (Batc  (None, 97, 97, 64)      

In [23]:
classifier.compile(
  optimizer='adam',
  loss="categorical_crossentropy",
  metrics=['accuracy'])


total_sample= train_generator.n
total_test=test_generator.n
batch_size=128


history= classifier.fit_generator(
            train_generator, steps_per_epoch=int(total_sample/batch_size),
            epochs=10,
            validation_data=test_generator,
            validation_steps=int(total_test/batch_size))

/var/folders/fs/rgrb722x0vxb50q4vybz6rnh0000gn/T/ipykernel_9924/2057428070.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history= classifier.fit_generator(


Epoch 1/10
 3/22 [===>..........................] - ETA: 2:30 - loss: 2.2098 - accuracy: 0.3724

KeyboardInterrupt: 

In [ ]:
classifier.save("model_2.h5")

In [ ]:
train_loss, train_acc = classifier.evaluate(train_generator)
print("\n Train Accuracy:", train_acc)
print("\n Train Loss:", train_loss)

In [25]:
from keras.models import load_model
model = load_model('model_2.h5')
 summarize model.
model.summary()
train_loss, train_acc = classifier.evaluate(train_generator)
print("\n Train Accuracy:", train_acc)
print("\n Train Loss:", train_loss)

23/23 [==============================] - 121s 5s/step - loss: 6.5042 - categorical_accuracy: 0.3211

 Train Accuracy: 0.32111993432044983

 Train Loss: 6.504210948944092


In [ ]:
# Test Loss/ Test Accuracy
test_loss, test_acc= classifier.evaluate(test_generator)
print("\n Test Accuracy:", test_acc)
print("\n Test Loss:", test_loss)

In [ ]:
classifier.metrics_names

In [ ]:
# PLOT ACCURACY
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model 2 Accuracy per Epoch')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower center')
plt.show()

In [ ]:
# PLOT LOSS
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model 2 Accuracy per Epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower center')
plt.show()

In [ ]:
# Save history of model into csv
hist_df2 = pd.DataFrame(history.history) 

# or save to csv: 
hist_csv_file = 'history2.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df2.to_csv(f)

# Model 3

In [ ]:
# Model 3
# GENERATE MODEL
classifier= Sequential()

# First convolution layer
classifier.add(Conv2D(32, (3,3), input_shape=(200,200,3),activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

# Second convolution layer
classifier.add(Conv2D(32,(3,3),activation="relu"))
classifier.add(Dropout(0.1))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

# Flatten the results to feed into a dense layer
classifier.add(Flatten())

# 128 neuron in the fully-connected layer
classifier.add(Dense(128 , activation = 'relu'))
classifier.add(Dropout(0.2))

# 14 output neurons for 14 classes with the softmax activation
classifier.add(Dense(3 , activation = 'softmax'))

In [ ]:
classifier.summary()

In [ ]:
# Model 3, with augmentation
classifier.compile(
  optimizer='adam',
  loss="categorical_crossentropy",
  metrics=['accuracy'])

# total_valsample=test_generator.n
total_sample= train_generator.n
total_test=test_generator.n
batch_size=128

history= classifier.fit_generator(
            train_generator, steps_per_epoch=int(total_sample/batch_size),
            epochs=10,
            validation_data=test_generator,
            validation_steps=int(total_test/batch_size))

In [ ]:
# Save Model-save weights
classifier.save("model_3.h5")

In [ ]:
# Train loss/ Train Accuracy
train_loss, train_acc = classifier.evaluate(train_generator)
print("\n Train Accuracy:", train_acc)
print("\n Train Loss:", train_loss)

In [ ]:
# Test Loss/ Test Accuracy
test_loss, test_acc= classifier.evaluate(test_generator)
print("\n Test Accuracy:", test_acc)
print("\n Test Loss:", test_loss)

In [ ]:
classifier.metrics_names

In [ ]:
# PLOT ACCURACY
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model 3 Accuracy per Epoch')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower center')
plt.show()

In [ ]:
# PLOT LOSS
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model 3 Accuracy per Epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower center')
plt.show()

In [ ]:
# Save history of model into csv
hist_df3 = pd.DataFrame(history.history) 

# or save to csv: 
hist_csv_file = 'history3.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df3.to_csv(f)

# Model 4- Transfer Learning

In [ ]:
img_width, img_height=200,200
batch_size=128

data_dir = "Database/"

# Rescale images 
datagen= ImageDataGenerator(rescale=1/255,validation_split=.3,rotation_range=20,
                           shear_range=.2,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,
                           preprocessing_function=preprocess_input)

# Flow training images in batches of 128 using train_data 
train_generator= datagen.flow_from_directory(
                data_dir,
                target_size=(64,64),
                batch_size=batch_size,
                subset="training",
                class_mode="categorical", 
                classes= ["COVID19", "NORMAL","Viral_Pneumonia"],
                shuffle=True, seed=30)


datagen2=ImageDataGenerator(rescale=1/255,validation_split=.3,
                            preprocessing_function=preprocess_input)

test_generator=datagen2.flow_from_directory(
                data_dir,
                target_size=(64,64),
                batch_size=batch_size,
                classes= ["COVID19", "NORMAL","Viral_Pneumonia"],
                class_mode="categorical", subset="validation", shuffle=True, seed=30)

In [ ]:
# Model 4

datagen= ImageDataGenerator(rescale=1/255,validation_split=.3)

classifier=VGG16(weights="imagenet", include_top=False, input_shape=(64,64,3))

for layer in classifier.layers:
    layer.trainable=False

x=Flatten()(classifier.output)
x=Dense(3, activation="softmax")(x)

model=Model(inputs=classifier.input, outputs=x)
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
total_sample= train_generator.n
total_test=test_generator.n
batch_size=128



history= model.fit_generator(
            train_generator, steps_per_epoch=int(total_sample/batch_size),
            epochs=100,
            validation_data=test_generator,
            validation_steps=int(total_test/batch_size))

In [ ]:
# Save Model-save weights
classifier.save("model_4tf_vgg16.h5")

In [ ]:
# Train loss/ Train Accuracy
train_loss, train_acc = model.evaluate(train_generator)
print("\n Train Accuracy:", train_acc)
print("\n Train Loss:", train_loss)

In [ ]:
# Test Loss/ Test Accuracy
test_loss, test_acc= model.evaluate(test_generator)
print("\n Test Accuracy:", test_acc)
print("\n Test Loss:", test_loss)

In [ ]:
model.metrics_names

In [ ]:
# PLOT ACCURACY
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model 3 Accuracy per Epoch')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower center')
plt.show()

In [ ]:
# PLOT LOSS
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model 3 Accuracy per Epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower center')
plt.show()

In [ ]:
# Save history of model into csv
hist_df4 = pd.DataFrame(history.history) 

# or save to csv: 
hist_csv_file = 'history4_tf_vgg16.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df4.to_csv(f)

## Model 5-Transfer Learning
### Same as Model 4 but at 200 epochs 

In [ ]:
# Model 5- Transfer Learning
data_dir = "Database/"
batch_size=128

# Rescale images 
datagen= ImageDataGenerator(rescale=1/255,validation_split=.3,rotation_range=20,
                           shear_range=.2,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,
                           preprocessing_function=preprocess_input)

# Flow training images in batches of 128 using train_data 
train_generator= datagen.flow_from_directory(
                data_dir,
                target_size=(64,64),
                batch_size=batch_size,
                subset="training",
                class_mode="categorical", 
                classes= ["COVID19", "NORMAL","Viral_Pneumonia"],
                shuffle=True, seed=30)


datagen2=ImageDataGenerator(rescale=1/255,validation_split=.3,
                            preprocessing_function=preprocess_input)

test_generator=datagen2.flow_from_directory(
                data_dir,
                target_size=(64,64),
                batch_size=batch_size,
                classes= ["COVID19", "NORMAL","Viral_Pneumonia"],
                class_mode="categorical", subset="validation", shuffle=True, seed=30)

In [ ]:
datagen= ImageDataGenerator(rescale=1/255,validation_split=.3)

# classifier=VGG16(weights="imagenet", include_top=False, input_shape=(64,64,3))


# x=Flatten()(classifier.output)
# x=Dense(3, activation="softmax")(x)


classifier = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(64, 64, 3)))

x = classifier.output
# x = AveragePooling2D(pool_size=(2, 2))(x)
x = Flatten(name="flatten")(x)
# x = Dense(128, activation="relu")(x)
# x = Dropout(0.6)(x)
x = Dense(3, activation="softmax")(x)

for layer in classifier.layers:
    layer.trainable=False

model=Model(inputs=classifier.input, outputs=x)
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=["accuracy"])

In [ ]:
total_sample= train_generator.n
total_test=test_generator.n
batch_size=128


history= model.fit_generator(
            train_generator, 
            steps_per_epoch=int(total_sample/batch_size),
            epochs=200,
            validation_data=test_generator,
            validation_steps=int(total_test/batch_size))

In [ ]:
# Save Model-save weights
classifier.save("model_5tf_vgg16.h5")

In [ ]:
# Train loss/ Train Accuracy
train_loss, train_acc = model.evaluate(train_generator)
print("\n Train Accuracy:", train_acc)
print("\n Train Loss:", train_loss)

In [ ]:
# Test Loss/ Test Accuracy
test_loss, test_acc= model.evaluate(test_generator)
print("\n Test Accuracy:", test_acc)
print("\n Test Loss:", test_loss)

In [ ]:
model.metrics_names

In [ ]:
# PLOT ACCURACY
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model 5 Accuracy per Epoch')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower center')
plt.show()

In [ ]:
# PLOT LOSS
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model 5 loss per Epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower center')
plt.show()

In [ ]:
# Save history of model into csv
hist_df5 = pd.DataFrame(history.history) 

# or save to csv: 
hist_csv_file = 'history5_tf_vgg16.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df5.to_csv(f)

In [ ]:
pip install pydot

In [ ]:
# tf.keras.utils.plot_model(
#     classifier, to_file='vgg16.png', show_shapes=False,
#     show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
# )


In [29]:
from keras.models import load_model
model = load_model('VGG16_model.h5')

model.summary()
train_loss, train_acc = classifier.evaluate(train_generator)
print("\n Train Accuracy:", train_acc)
print("\n Train Loss:", train_loss)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 3, 3, 512)        0         
 2D)                                                             
                                                                 
 dense_6 (Dense)             (None, 3, 3, 224)         114912    
                                                                 
 flatten_3 (Flatten)         (None, 2016)              0         
                                                                 
 dense_7 (Dense)             (None, 3)                 6051      
                                                                 
Total params: 14,835,651
Trainable params: 120,963
Non-trainable params: 14,714,688
____________________________________